In [18]:
### Imports

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd

## 0. Data Visualization

In [14]:
df = pd.read_csv('../DL-2025-CW-data/Data_A.csv', names=['target', 'lvl1_sell_p', 'lvl1_sell_vol', 'lvl1_buy_p', 'lvl1_buy_vol', 'lvl2_sell_p', 'lvl2_sell_vol', 'lvl2_buy_p', 'lvl2_buy_vol', 'lvl3_sell_p', 'lvl3_sell_vol', 'lvl3_buy_p', 'lvl3_buy_vol', 'lvl4_sell_p', 'lvl4_sell_vol', 'lvl4_buy_p', 'lvl4_buy_vol', 'prev_midprice1', 'prev_midprice2', 'prev_midprice3', 'prev_midprice4', 'prev_midprice5'])
df.head()

,target,lvl1_sell_p,lvl1_sell_vol,lvl1_buy_p,lvl1_buy_vol,lvl2_sell_p,lvl2_sell_vol,lvl2_buy_p,lvl2_buy_vol,lvl3_sell_p,...,lvl3_buy_vol,lvl4_sell_p,lvl4_sell_vol,lvl4_buy_p,lvl4_buy_vol,prev_midprice1,prev_midprice2,prev_midprice3,prev_midprice4,prev_midprice5
0,1,693200.0,126,692800.0,110,693300.0,50,692700.0,165,693400.0,...,50,693500.0,50,692500.0,250,0,1,0,1,0
1,0,650400.0,501,650200.0,106,650500.0,245,650100.0,259,650600.0,...,328,650700.0,141,649900.0,277,0,0,1,1,1
2,1,636200.0,153,635800.0,150,636300.0,100,635700.0,15,636400.0,...,110,636500.0,105,635500.0,201,0,1,0,1,0
3,0,724800.0,4,724500.0,14,724900.0,50,724300.0,312,725100.0,...,100,725200.0,379,724100.0,56,1,0,0,1,1
4,0,622900.0,110,622700.0,100,623000.0,523,622600.0,300,623100.0,...,249,623200.0,605,622400.0,200,1,0,1,1,0


In [20]:
bid_orders = df.loc[1,:].filter(like='buy_p')
ask_orders = df.loc[1,:].filter(like='sell_p')
bid_vol = df.loc[1,:].filter(like='buy_vol')
ask_vol = df.loc[1,:].filter(like='sell_vol')